In [3]:
from eia_api_client import EIAClient, Frequency, DataType, Facet
import json
from datetime import datetime, timezone, timedelta

In [4]:
with open("config.json", "r") as f:
    config_json = json.load(f)

api_key = config_json.get("api_key")

eia_client = EIAClient(api_key)
eia_response = eia_client.get_data(
    frequency=Frequency.LOCAL_HOURLY,
    data_type=DataType.VALUE,
    facet=Facet.MIDA,
    start=datetime(2023, 6, 24, tzinfo=timezone(offset=-timedelta(hours=4))),
    end=datetime(2023, 7, 1, tzinfo=timezone(offset=-timedelta(hours=4)))
)

In [10]:
data = eia_response.data
data[:10]

[Data(period='2023-06-23T20-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=98464, value_units='megawatthours'),
 Data(period='2023-06-23T21-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=96522, value_units='megawatthours'),
 Data(period='2023-06-23T22-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=94688, value_units='megawatthours'),
 Data(period='2023-06-23T23-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=90957, value_units='megawatthours'),
 Data(period='2023-06-24T00-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=86713, value_units='megawatthours'),
 Data(period='2023-06-24T01-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=82649, value_units='megawatthours'),
 Data(period='2023-06-24T02-04', respondent='MIDA', respon

In [13]:
demand_data = list(filter(lambda x: x.type == "D", data))
demand_data

[Data(period='2023-06-23T20-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=98464, value_units='megawatthours'),
 Data(period='2023-06-23T21-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=96522, value_units='megawatthours'),
 Data(period='2023-06-23T22-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=94688, value_units='megawatthours'),
 Data(period='2023-06-23T23-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=90957, value_units='megawatthours'),
 Data(period='2023-06-24T00-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=86713, value_units='megawatthours'),
 Data(period='2023-06-24T01-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=82649, value_units='megawatthours'),
 Data(period='2023-06-24T02-04', respondent='MIDA', respon

In [14]:
import plotly.express as px

fig = px.bar(x=[data_item.period for data_item in demand_data], y=[data_item.value for data_item in demand_data])
fig.show()

In [8]:
[data_item for data_item in data if data_item.period == "2023-06-24T05-04"]

[Data(period='2023-06-24T05-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=74984, value_units='megawatthours'),
 Data(period='2023-06-24T05-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='DF', type_name='Day-ahead demand forecast', value=68913, value_units='megawatthours'),
 Data(period='2023-06-24T05-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='NG', type_name='Net generation', value=80352, value_units='megawatthours'),
 Data(period='2023-06-24T05-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='TI', type_name='Total interchange', value=5374, value_units='megawatthours')]